In [2]:
import torch

In [3]:
torch.cuda.is_available()

True

In [3]:
!nvidia-smi

Failed to initialize NVML: Unknown Error


In [4]:
import os
import psutil
torch.cuda.set_device(0) 
torch.backends.cudnn.benchmark = True 
num_workers = os.cpu_count()
torch.set_num_threads(num_workers)
ram_gb = psutil.virtual_memory().total / (1024 * 1024 * 1024)
batch_size = int(ram_gb / 2) 

RuntimeError: No CUDA GPUs are available

In [5]:
batch_size, num_workers

(55, 16)

In [4]:
import psutil
def monitor_resources(): # killer ficha NENADO
    while True:
        print(f"CPU Usage: {psutil.cpu_percent()}%")
        print(f"Memory Usage: {psutil.virtual_memory().percent}%")
        time.sleep(1)


In [6]:
import time

In [8]:
import subprocess
print(f"GPU count: {torch.cuda.device_count()}")
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")


GPU count: 1
GPU 0: NVIDIA L40S


In [9]:
pip install memory_profiler

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [ ]:
def log_progress(sequence, every=10):
    from ipywidgets import IntProgress
    from IPython.display import display

    progress = IntProgress(min=0, max=len(sequence), value=0)
    display(progress)
    
    for index, record in enumerate(sequence):
        if index % every == 0:
            progress.value = index
        yield record

## Sentence transformation

In [16]:
# Load model directly
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("sergeyzh/LaBSE-ru-turbo")
model = AutoModel.from_pretrained("sergeyzh/LaBSE-ru-turbo")

In [18]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sergeyzh/LaBSE-ru-turbo") #вроде норм с ин яз

sentences = [
    "Это счастливый человек",
    "Это счастливая собака",
    "Это очень счастливый человек",
    "Сегодня солнечный день"
]
embeddings = model.encode(sentences)

similarities = model.similarity(embeddings, embeddings)
print(similarities)
embeddings

tensor([[1.0000, 0.8071, 0.9659, 0.6278],
        [0.8071, 1.0000, 0.7807, 0.5681],
        [0.9659, 0.7807, 1.0000, 0.6036],
        [0.6278, 0.5681, 0.6036, 1.0000]])


array([[ 0.0303947 , -0.00513429, -0.01320625, ..., -0.00933791,
         0.00857514,  0.01643607],
       [ 0.0395426 ,  0.02917142, -0.02478084, ...,  0.02451655,
         0.01751278,  0.04843901],
       [ 0.02484259, -0.00738657, -0.02211446, ..., -0.00596734,
        -0.00914622,  0.00237997],
       [-0.00288085, -0.01277977, -0.00787543, ...,  0.02841871,
        -0.01064172,  0.01239938]], dtype=float32)

Multilingual

In [9]:
#pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(sentences)
print(embeddings)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[ 0.18845685  0.17425634  0.05447784  0.29051757  0.1676642  -0.04720681
   0.64558     0.15980887  0.22689249 -0.03089052  0.25588349 -0.05258771
  -0.22610143 -0.05710632  0.13042632  0.12495351  0.31749603  0.19444391
  -0.5863255  -0.01258594  0.6099092   0.1643273   0.03331115 -0.27383074
  -0.28975755 -0.21119712 -0.02261393 -0.17035922  0.16159014  0.06082741
  -0.24162416  0.18579192  0.4274095   0.19295181 -0.07234461  0.16611089
   0.10442799  0.20477238  0.21116723  0.1997399  -0.09408264 -0.17383674
   0.06427359  0.28025466 -0.29530582  0.06209537  0.1042769  -0.02364421
   0.12913182 -0.12617469 -0.17899004  0.0370057  -0.61250603  0.05029817
   0.17730355  0.22494122  0.17386058 -0.03840296 -0.21286812  0.25849253
  -0.12101638  0.30971512 -0.4196637   0.00907664  0.14188902 -0.30556944
   0.17621145 -0.07087357 -0.62033135  0.6770835   0.0172375   0.18405119
  -0.16785784  0.20452644 -0.14770274 -0.06175363  0.6301742   0.11120176
   0.0515307   0.15927427 -0.05370893 

## Zero-shot text classification

In [6]:
#!pip install transformers sentencepiece --quiet
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_checkpoint = 'cointegrated/rubert-base-cased-nli-threeway'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model.cuda()

text1 = 'Сократ - человек, а все люди смертны.'
text2 = 'Сократ никогда не умрёт.'
with torch.inference_mode():
    out = model(**tokenizer(text1, text2, return_tensors='pt').to(model.device))
    proba = torch.softmax(out.logits, -1).cpu().numpy()[0]
print({v: proba[k] for k, v in model.config.id2label.items()})
# {'entailment': 0.009525929, 'contradiction': 0.9332064, 'neutral': 0.05726764}


tokenizer_config.json:   0%|          | 0.00/545 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

2024-11-09 19:06:26.977994: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-09 19:06:26.978041: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-09 19:06:26.979301: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-09 19:06:27.074705: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-09 19:06:30.093669: W tensorflow/comp

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

{'entailment': 0.009525914, 'contradiction': 0.9332064, 'neutral': 0.057267707}


In [7]:
def predict_zero_shot(text, label_texts, model, tokenizer, label='entailment', normalize=True):
    label_texts
    tokens = tokenizer([text] * len(label_texts), label_texts, truncation=True, return_tensors='pt', padding=True)
    with torch.inference_mode():
        result = torch.softmax(model(**tokens.to(model.device)).logits, -1)
    proba = result[:, model.config.label2id[label]].cpu().numpy()
    if normalize:
        proba /= sum(proba)
    return proba
classes = ['Я доволен', 'Я недоволен']
predict_zero_shot('Какая гадость эта ваша заливная рыба!', classes, model, tokenizer)
# array([0.05609814, 0.9439019 ], dtype=float32)
predict_zero_shot('Какая вкусная эта ваша заливная рыба!', classes, model, tokenizer)
# array([0.9059292 , 0.09407079], dtype=float32)


def predict_zero_shot_df(df, classes, model, tokenizer, label='entailment', normalize=True):
    predictions = []
    for text in df['text']:  
        tokens = tokenizer([text] * len(classes), classes, truncation=True, return_tensors='pt', padding=True)
        with torch.inference_mode():
            result = torch.softmax(model(**tokens.to(model.device)).logits, -1)
        proba = result[:, model.config.label2id[label]].cpu().numpy()
        if normalize:
            proba /= sum(proba)
        # Получаем класс с максимальной вероятностью
        pred_class = classes[proba.argmax()]
        predictions.append(pred_class)
    return predictions

# preds = predict_zero_shot_df(test, df.target.unique(), model, tokenizer)
# subm['target'] = preds

array([0.9059289 , 0.09407106], dtype=float32)

English

In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")
sequence_to_classify = "one day I will see the world"
candidate_labels = ['travel', 'cooking', 'dancing']
classifier(sequence_to_classify, candidate_labels)
#{'labels': ['travel', 'dancing', 'cooking'],
# 'scores': [0.9938651323318481, 0.0032737774308770895, 0.002861034357920289],
# 'sequence': 'one day I will see the world'}

### Полезные ссылки

- https://huggingface.co/ai-forever/sbert_large_nlu_ru
- https://huggingface.co/cointegrated/rubert-tiny2 
- https://github.com/Vlad15lav/food-recsys/blob/main/notebooks/demo_bert_recsys.ipynb
- https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.ZeroShotClassificationPipeline
